In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [145]:
import importlib
import ensemble
importlib.reload(ensemble)

<module 'ensemble' from '/Users/oliver/Projects/dl-cyclones/ensemble/ensemble.py'>

In [110]:
# Generate high dimensional Gaussian noise with nonlinear relationships between the source and target vectors

import math

def get_data_point():
#     latent = np.random.randn(10)
#     noise = np.random.randn(2) * latent[3] + latent[5]/2
#     target = np.array([
#         latent[0] + latent[1]**2 - 10 * math.sqrt(abs(latent[4])),
#         latent[7]**3 - 7 * latent[0]
#     ]) + noise
#     source = np.resize(latent, 100) * 5 - 10 * np.resize(latent**2, 100)
    source = np.random.randn(100)
    noise = np.random.randn(2)
    target = np.array([
        source[0:10].mean() + (source[11:20]**2).mean() / (source[21:30].mean() + 1) - np.abs(source[31:40]).mean(),
        source[7:21].mean() + (source[11:20:2]**2).mean() / (source[27:33].mean() + 1) - np.abs(source[35:47]).mean(),
    ]) + noise
    return source, target, noise

def transpose_tuples(arr):
    xs = []
    ys = []
    zs = []
    for x, y, z in arr:
        xs.append(x)
        ys.append(y)
        zs.append(z)
    return np.array(xs), np.array(ys), np.array(zs)

# each row is the feature vector for a time interval
train_feature_vectors, train_label_vectors, train_noise = transpose_tuples([get_data_point() for i in range(200)])
print(train_feature_vectors.shape, train_label_vectors.shape)

valid_feature_vectors, valid_label_vectors, valid_noise = transpose_tuples([get_data_point() for i in range(50)])
print(valid_feature_vectors.shape, valid_label_vectors.shape)

(200, 100) (200, 2)
(50, 100) (50, 2)


In [119]:
# Bound the loss
min_loss = haversine_loss(train_label_vectors, train_label_vectors - train_noise).mean()
min_loss

145.56469228287506

In [146]:
# train all of the models in model_classes on the training data
def train(model_class):
    model = model_class()
    print("\n\nTraining model " + model.NAME)
    model.train(train_feature_vectors, valid_feature_vectors, train_label_vectors, valid_label_vectors, verbose=True)
    print("Saving model " + model.NAME + " which had validation mean km error of " + str(model.mean_km))
    model.save(model.NAME)
# for model_class in ensemble.model_classes: train_model(model_class)

In [147]:
train(ensemble.ANNModel)



Training model ANN
Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_86 (Dense)            (None, 256)               25856     
                                                                 
 batch_normalization_56 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 dropout_58 (Dropout)        (None, 256)               0         
                                                                 
 dense_87 (Dense)            (None, 256)               65792     
                                                                 
 batch_normalization_57 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 dropout_59 (Dropout)        (No

2022-05-09 21:25:11.760178: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 198ms/step - loss: 238.2637 - val_loss: 188.3912
Epoch 2/10
2/2 [==============================] - 0s 58ms/step - loss: 227.0731 - val_loss: 188.7443
Epoch 3/10


2022-05-09 21:25:12.200181: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 0s 38ms/step - loss: 191.9930 - val_loss: 188.6539
Epoch 4/10
2/2 [==============================] - 0s 42ms/step - loss: 187.1098 - val_loss: 188.0447
Epoch 5/10
2/2 [==============================] - 0s 35ms/step - loss: 177.2401 - val_loss: 187.6818
Epoch 6/10
2/2 [==============================] - 0s 37ms/step - loss: 175.0824 - val_loss: 187.4015
Epoch 7/10
2/2 [==============================] - 0s 33ms/step - loss: 160.4684 - val_loss: 186.9798
Epoch 8/10
2/2 [==============================] - 0s 28ms/step - loss: 165.1616 - val_loss: 186.6308
Epoch 9/10
2/2 [==============================] - 0s 30ms/step - loss: 146.8151 - val_loss: 186.3603
Epoch 10/10
2/2 [==============================] - 0s 30ms/step - loss: 152.5287 - val_loss: 186.3961
validation mean km error: 186.4
Saving model ANN which had validation mean km error of 186.39609945199902


2022-05-09 21:25:12.830054: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [148]:
train(ensemble.AdaBoostModel)



Training model ADA
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
fit models
[{'n_estimators': 300}, {'n_estimators': 300}]
fit best model
validation mean km error: 182.2
Saving model ADA which had validation mean km error of 182.1586788573517


In [149]:
train(ensemble.KNNModel)



Training model KNN
Fitting 5 folds for each of 3 candidates, totalling 15 fits
fit models
fit best model
validation mean km error: 184.3
Saving model KNN which had validation mean km error of 184.31054377885778


In [150]:
train(ensemble.RandomForestModel)



Training model RF
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

fit models
fit best model
validation mean km error: 186.2
Saving model RF which had validation mean km error of 186.20463279667783


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished


In [152]:
train(ensemble.XGBModel)



Training model XGB
[21:25:24] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/learner.cc:627: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


fit model
validation mean km error: 203.1
Saving model XGB which had validation mean km error of 203.14008209110415


In [153]:
# Save/load models, check they work after loading from disk
models = [model_class.load(model_class.NAME) for model_class in ensemble.model_classes]

In [154]:
print(f"Min possible loss: {min_loss:.2f}")
for model in models:
    print(f"{model.NAME:<8} val loss: {model.mean_km:.2f}")

Min possible loss: 145.56
ANN      val loss: 186.40
ADA      val loss: 182.16
KNN      val loss: 184.31
RF       val loss: 186.20
XGB      val loss: 203.14


In [164]:
# Ensemble these models together and see how the ensemble loss compares
preds = np.array([model.predict(valid_feature_vectors) for model in models])
print([model.NAME for model in models])
print([preds[i].shape for i in range(len(preds))])
preds.shape

['ANN', 'ADA', 'KNN', 'RF', 'XGB']
[(50, 2), (50, 2), (50, 2), (50, 2), (50, 2)]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished


(5, 50, 2)

In [165]:
ensemble_preds = np.swapaxes(np.swapaxes(preds, 0, 2), 0, 1).mean(axis=2)
haversine_loss(ensemble_preds, valid_label_vectors).mean()

180.51402616749732